<a href="https://colab.research.google.com/github/sh-mukherjee/World-Bank-Renew-Energy/blob/master/World_Bank_Renewable_Energy_Consumption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Introduction


In this notebook, we will be working with the Renewable Energy Consumption (% Of Total Final Energy Consumption) dataset from the World Bank. We will perform analysis on this data using visualizations with Plotly Express and JupyterDash (the Dash web application library adapted for use in Jupyter notebooks)

 **Attribution**

[The World Bank](https://datacatalog.worldbank.org)

**Dataset Name**: [Renewable Energy Consumption (% of Total Final Energy Consumption)](https://databank.worldbank.org/reports.aspx?source=2&series=EG.FEC.RNEW.ZS)

**Data Source**:
World Bank, Sustainable Energy for All (SE4ALL) database from the SE4ALL Global Tracking Framework led jointly by the World Bank, International Energy Agency, and the Energy Sector Management Assistance Program.

The dataset (zipped file) can be downloaded in csv or Excel format. I downloaded and saved the csv version to my Google drive, to which I have provided a link in the notebook.

[Zipped data file download link from Worldbank](https://datacatalog.worldbank.org/%23)

The World Bank has its own visualisation tool for this data, but I did not find it very satisfactory. Therefore we'll visualise it ourselves in this notebook.

[World Bank's visualisation tool](https://databank.worldbank.org/reports.aspx?source=2&series=EG.FEC.RNEW.ZS&country=)

The Workd Bank doesn't state explicitly what are counted as  renewable energy sources in this dataset, but from the descriptions for other World Bank datasets, they seem to include geothermal, solar, tides, wind, biomass, biofuels, and hydroelectric. 

It will be interesting to see which countries use more renewable energy, and what kind of trend can be discerned in different regions.

# Installing and importing the required packages

In [15]:
!pip install jupyter-dash #install the Jupyter Dash library. We will need this for the final visualisation

In [16]:
import pandas as pd 
#pandas is a library written for the Python programming language for data manipulation and analysis. Check it out at https://pandas.pydata.org/
import plotly.express as px 
#high level Python library for data visualisation. Check it out at https://plotly.com/python/plotly-express/
from jupyter_dash import JupyterDash 
#JupyterDash is the Jupyter-friendly version of Dash, which is a productive Python framework for building web applications. Check it out at https://dash.plotly.com/
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Loading and exploring the data

In [17]:
url_view = 'https://drive.google.com/file/d/16YGHVk6p6F7mAMikowqg8CCSeNUsLHc4/view?usp=sharing' 
#Link to the unzipped csv file (with top description lines removed) stored in my Google drive. Anybody with this link can view the csv data file.
path = 'https://drive.google.com/uc?export=download&id='+url_view.split('/')[-2] #path to access the file

#read in the csv file into a dataframe
df = pd.read_csv(path)

# Checking the type of data that we get by looking at the first few rows
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,Renewable energy consumption (% of total final...,EG.FEC.RNEW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.268747,0.234955,0.238778,0.195134,0.183241,0.173372,0.167474,0.160096,0.157876,0.160928,0.175266,0.180523,0.181391,0.184550,0.187055,0.186599,0.189937,0.190059,0.193546,0.299774,5.464716,5.661788,6.855850,6.889753,6.927502,6.726748,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,Renewable energy consumption (% of total final...,EG.FEC.RNEW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.924532,17.036444,26.521629,30.585667,32.796251,35.075640,37.945748,41.432601,44.094337,52.185774,54.243126,54.055055,43.771149,42.276141,49.843148,40.859171,37.137249,33.862579,21.343708,17.813855,14.839806,11.482706,13.973586,16.334293,19.314269,18.423477,NaN,NaN,NaN,NaN,NaN
2,Angola,AGO,Renewable energy consumption (% of total final...,EG.FEC.RNEW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.255252,71.888639,72.723283,71.277056,72.220269,73.945888,73.139600,73.734416,76.819794,73.410784,74.618171,73.757859,72.125126,67.306117,65.493065,70.954202,65.022564,61.599704,58.107975,55.748977,54.193837,52.715679,52.245736,50.686116,50.797461,49.568210,NaN,NaN,NaN,NaN,NaN
3,Albania,ALB,Renewable energy consumption (% of total final...,EG.FEC.RNEW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.518088,32.998174,46.813329,51.152042,51.459544,50.609710,51.636361,55.953349,49.983365,42.718313,41.445416,39.125664,35.896294,33.752729,35.935868,36.869489,31.710195,32.100937,35.912906,37.216638,37.115330,35.962532,40.048300,41.288974,38.689501,38.615210,NaN,NaN,NaN,NaN,NaN
4,Andorra,AND,Renewable energy consumption (% of total final...,EG.FEC.RNEW.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.273550,14.273550,14.308754,13.918955,14.561024,14.481048,14.100048,14.194002,14.458642,14.549891,14.890664,15.773868,16.221819,16.912308,16.874428,16.902437,17.485994,16.940777,17.422741,17.515948,19.090730,18.971546,19.195529,19.563698,19.886323,19.747809,NaN,NaN,NaN,NaN,NaN


This dataframe is very wide because individual years appear as separate columns. The years 1960 to 1989, as well as the years from 2016 onwards do not seem to have any data. In order to verify this, let us have a look at the information about the dataframe's columns, rows and values.


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 65 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    264 non-null    object 
 1   Country Code    264 non-null    object 
 2   Indicator Name  264 non-null    object 
 3   Indicator Code  264 non-null    object 
 4   1960            0 non-null      float64
 5   1961            0 non-null      float64
 6   1962            0 non-null      float64
 7   1963            0 non-null      float64
 8   1964            0 non-null      float64
 9   1965            0 non-null      float64
 10  1966            0 non-null      float64
 11  1967            0 non-null      float64
 12  1968            0 non-null      float64
 13  1969            0 non-null      float64
 14  1970            0 non-null      float64
 15  1971            0 non-null      float64
 16  1972            0 non-null      float64
 17  1973            0 non-null      flo

# Tidying the Data

As expected, there are a number of year columns with no data. We will now tidy up the dataframe. Firstly, it does not make sense to keep those year columns that do not have any non-null values, so we will drop them.

In [19]:
#drop the columns which do not have ANY non-null values
df = df.dropna(axis=1, how='all')
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,Renewable energy consumption (% of total final...,EG.FEC.RNEW.ZS,0.268747,0.234955,0.238778,0.195134,0.183241,0.173372,0.167474,0.160096,0.157876,0.160928,0.175266,0.180523,0.181391,0.184550,0.187055,0.186599,0.189937,0.190059,0.193546,0.299774,5.464716,5.661788,6.855850,6.889753,6.927502,6.726748
1,Afghanistan,AFG,Renewable energy consumption (% of total final...,EG.FEC.RNEW.ZS,15.924532,17.036444,26.521629,30.585667,32.796251,35.075640,37.945748,41.432601,44.094337,52.185774,54.243126,54.055055,43.771149,42.276141,49.843148,40.859171,37.137249,33.862579,21.343708,17.813855,14.839806,11.482706,13.973586,16.334293,19.314269,18.423477
2,Angola,AGO,Renewable energy consumption (% of total final...,EG.FEC.RNEW.ZS,72.255252,71.888639,72.723283,71.277056,72.220269,73.945888,73.139600,73.734416,76.819794,73.410784,74.618171,73.757859,72.125126,67.306117,65.493065,70.954202,65.022564,61.599704,58.107975,55.748977,54.193837,52.715679,52.245736,50.686116,50.797461,49.568210
3,Albania,ALB,Renewable energy consumption (% of total final...,EG.FEC.RNEW.ZS,25.518088,32.998174,46.813329,51.152042,51.459544,50.609710,51.636361,55.953349,49.983365,42.718313,41.445416,39.125664,35.896294,33.752729,35.935868,36.869489,31.710195,32.100937,35.912906,37.216638,37.115330,35.962532,40.048300,41.288974,38.689501,38.615210
4,Andorra,AND,Renewable energy consumption (% of total final...,EG.FEC.RNEW.ZS,14.273550,14.273550,14.308754,13.918955,14.561024,14.481048,14.100048,14.194002,14.458642,14.549891,14.890664,15.773868,16.221819,16.912308,16.874428,16.902437,17.485994,16.940777,17.422741,17.515948,19.090730,18.971546,19.195529,19.563698,19.886323,19.747809


It looks better now, but the data is still not in an ideal format. The 'Indicator Name' and 'Indicator Code' columns are not really needed, since there is only thing that is being measured here -- the 'Renewable energy consumption (% of total final energy consumption)'. 



In [20]:
cols = ['Indicator Name', 'Indicator Code'] #making a list of column names that will be dropped
df = df.drop(cols, axis=1)
df.head()

,Country Name,Country Code,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,0.268747,0.234955,0.238778,0.195134,0.183241,0.173372,0.167474,0.160096,0.157876,0.160928,0.175266,0.180523,0.181391,0.184550,0.187055,0.186599,0.189937,0.190059,0.193546,0.299774,5.464716,5.661788,6.855850,6.889753,6.927502,6.726748
1,Afghanistan,AFG,15.924532,17.036444,26.521629,30.585667,32.796251,35.075640,37.945748,41.432601,44.094337,52.185774,54.243126,54.055055,43.771149,42.276141,49.843148,40.859171,37.137249,33.862579,21.343708,17.813855,14.839806,11.482706,13.973586,16.334293,19.314269,18.423477
2,Angola,AGO,72.255252,71.888639,72.723283,71.277056,72.220269,73.945888,73.139600,73.734416,76.819794,73.410784,74.618171,73.757859,72.125126,67.306117,65.493065,70.954202,65.022564,61.599704,58.107975,55.748977,54.193837,52.715679,52.245736,50.686116,50.797461,49.568210
3,Albania,ALB,25.518088,32.998174,46.813329,51.152042,51.459544,50.609710,51.636361,55.953349,49.983365,42.718313,41.445416,39.125664,35.896294,33.752729,35.935868,36.869489,31.710195,32.100937,35.912906,37.216638,37.115330,35.962532,40.048300,41.288974,38.689501,38.615210
4,Andorra,AND,14.273550,14.273550,14.308754,13.918955,14.561024,14.481048,14.100048,14.194002,14.458642,14.549891,14.890664,15.773868,16.221819,16.912308,16.874428,16.902437,17.485994,16.940777,17.422741,17.515948,19.090730,18.971546,19.195529,19.563698,19.886323,19.747809


The dataframe looks pretty good now, but it is still quite 'wide'.
Ideally we should have one column for the country name, one column for the country code (which is in ISO Alpha format and thus will be helpful for geographical charts), one column for the year (which will have values from 1990 to 2015), and one column for the renewable energy consumption as a percentage of the total final energy consumption.

The way to accomplish this is to 'melt' this dataframe so that it becomes 'long-form'.

In [21]:
df_melt = pd.melt(df, id_vars=['Country Name','Country Code'],var_name='Year',value_name='Renewable Energy Cons % of Total Energy Cons')
df_melt.head()

,Country Name,Country Code,Year,Renewable Energy Cons % of Total Energy Cons
0,Aruba,ABW,1990,0.268747
1,Afghanistan,AFG,1990,15.924532
2,Angola,AGO,1990,72.255252
3,Albania,ALB,1990,25.518088
4,Andorra,AND,1990,14.273550


In [22]:
df_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6864 entries, 0 to 6863
Data columns (total 4 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Country Name                                  6864 non-null   object 
 1   Country Code                                  6864 non-null   object 
 2   Year                                          6864 non-null   object 
 3   Renewable Energy Cons % of Total Energy Cons  6588 non-null   float64
dtypes: float64(1), object(3)
memory usage: 214.6+ KB


The Year value is being read as a string. We will change it to a number to enable us to choose minimum and maximum values for future use.

In [23]:
df_melt['Year'] = pd.to_numeric(df_melt['Year'])
df_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6864 entries, 0 to 6863
Data columns (total 4 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Country Name                                  6864 non-null   object 
 1   Country Code                                  6864 non-null   object 
 2   Year                                          6864 non-null   int64  
 3   Renewable Energy Cons % of Total Energy Cons  6588 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 214.6+ KB


In [24]:
#Writing the cleaned dataframe into a csv file in order to use in future projects. You can save this file into your preferred location.
df_melt.to_csv('renewenergy.csv', index=False, header=True) 


# Data Visualisation

Now let us plot the data! We will use Plotly Express for the plotting.

In [25]:
fig=px.line(data_frame=df_melt,x='Year',y='Renewable Energy Cons % of Total Energy Cons',color='Country Name',width=1200, height=600) #plotting a line chart for all the countries and all the years
fig.show()

You can hover over the lines in order to see the name of the country and the percentage of renewable energy consumed. However, this chart is too crowded and it is nearly impossible to discern any individual country. It would be nice to have some way of choosing specfic countries and years and looking at the data for those. This is where the JupyterDash library comes in. We will create a simple interactive dashboard with dropdowns and sliders where you can select one or more countries and look at their line charts, as well as choose a year and look at the resulting geo heat map.

## Dashboard with a Line Chart and Map Chart

This dashboard will have two visualizations -- the first is a line chart that shows the yearly percentages of renewable energy in the total energy consumption for the countries chosen from a dropdown, and the second is a map showing the percentages of renewable energy in the total energy consumption for all the countries in the world, for any particular year chosen from a year selection slider.

In [26]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'] #this is a commonly used CSS to set the styles for the output

app = JupyterDash(__name__,external_stylesheets=external_stylesheets)

app.layout = html.Div([
   html.Div([
    dcc.Graph(id='country-level'),
    dcc.Dropdown(
        id='country-drop',
        options=[
                  {'label': i, 'value': i} for i in df_melt['Country Name'].unique()] #setting the values in the dropdown list
        ,
        value=['United Kingdom', 'United States', 'Japan', 'Germany', 'Australia', 'China', 'India'], #setting some pre-selected values to start off with
        multi=True  #ability to select multiple items from the dropdown
    )]),
    html.Div([
    dcc.Graph(id = 'year-level'),                    
    dcc.Slider(
        id='year-slider',
        min=df_melt['Year'].min(),
        max=df_melt['Year'].max(),
        value = df_melt['Year'].max(),
        marks = {str(year): str(year) for year in df_melt['Year'].unique()}
    )])
])


@app.callback(
    [Output('country-level', 'figure'), Output('year-level', 'figure')],
    [Input('country-drop', 'value'), Input('year-slider', 'value')])

def update_figure(selected_country, selected_year):
    filtered_df = df_melt[df_melt['Country Name'].isin(selected_country)] #creates a new dataframe filtered on the countries chosen from the dropdown

    fig = px.line(filtered_df, x="Year", y="Renewable Energy Cons % of Total Energy Cons", 
                     color="Country Name", width = 1100)  #plots an interactive line chart with the data filtered on the country name

    fig.update_layout(transition_duration=300)

    filt_df = df_melt[df_melt['Year'] == selected_year] #creates a new dataframe filtered on the year chosen from the slider

    figmap = px.choropleth(filt_df,               
              locations="Country Code",               
              color="Renewable Energy Cons % of Total Energy Cons",
              hover_name="Country Name",   
              color_continuous_scale='Greens', width=1100, height = 500) #plots a chloropleth chart with the data filtered on the year

    figmap.update_layout(transition_duration=300)

    return fig, figmap

# Run app and display result inline in the notebook
if __name__ == '__main__':
    app.run_server(mode='inline')

<IPython.core.display.Javascript object>

# Concluding Remarks

Play around with the countries and the years to see what percentage of the total energy consumption was from renewable energy sources.

I'll just point out some observations below. Keep in mind that the data only spans 1990 to 2015, so there may have been significant developments in the last five years!

* Richer, developed countries generally have a low 
percentage of their total energy coming from renewable sources, though some of them show an increasing trend.

* Norway and Iceland, who make use of geothermal energy, have a higher proportion of their consumption coming from renewable energy.

* Many poorer and less-developed countries have MUCH higher levels of renewable energy consumption as a percentage of their total energy consumption, see Nepal for example.

* This could be due to a combination of hydroelectric power for industrial power generation and the use of biomass for domestic energy needs.

* The line charts for India and China show a decreasing trend over the years as they have grown richer, perhaps due to rapidly increasing energy demand that cannot be satisfied fast enough by renewable energy.

All of this gives us a lot to think about. We should keep in mind that 'renewable' does not necessarily mean 'clean' or 'sustainable' or 'environmentally friendly'.
Wood is technically 'renewable', but using too much of it as fuel could lead to deforestation, and domestic use for cooking produces indoor pollution that has negative effects on the health of household members. Hydroelectric power also has its own challenges with respect to negative environmental consequences.

But we shouldn't be too quick to dismiss the data on this basis. Many developing countries are investing in better renewable energy technology nowadays.

The [Renewables Global Status Report](https://ren21.net/gsr-2019/) (GSR), released annually by the Renewable Energy Policy Network for the 21st Century (REN21, a think tank) is a detailed report on the the different sources of renewable energy, their growth rates, and how various countries are using renewable energy.